Load extrated data

In [ ]:
%run twitter_api.ipynb

In [ ]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib
from nltk import FreqDist
from fuzzywuzzy import process

Extraindo as hashtags e guardando em uma lista

In [ ]:
data = df_tweets.hashtags.apply(lambda x: np.nan if len(x) <= 0 else x)
all_hashtags = list(data.dropna())
all_hashtags[0:5]

Análise de frequências de hashtags

In [ ]:
hashtags = []
for i in all_hashtags:
    for j in i:
        hashtags.append(j)

In [ ]:
hashtags[0:10]

In [ ]:
hash_str = ''

for i in hashtags:
    hash_str += i + ' '

hash_str = hash_str.lower()
hashtags2 = hash_str.split()

In [ ]:
hashtags2[0:10]

In [ ]:
freq = FreqDist(hashtags2)
hash_most_freq = pd.DataFrame(data = freq.most_common(10), columns = ['Hashtag', 'Frequency'])
hash_most_freq

In [ ]:
list_freq = list(hash_most_freq.Hashtag)
list_freq

Funçao para comparar todas as hashtags dos tweets com a lista das mais frequentes

In [ ]:
def select_hashtag(freq, all_hash):
    select = []
    
    for list_hash in all_hash:
        for f in freq:
            if (len(list_hash) >=2 and (f in list_hash)):
                select.append(list_hash)
                break
            else:
                pass
    return select

In [ ]:
""" def select_hashtag(freq, all_hash):
    select = []
    for f in freq:
        for list_hash in all_hash:
            if len(list_hash) >=2:
                highest = process.extractOne(f, list_hash)
                if highest[1] > 90:
                    select.append(list_hash)
                else:
                    pass
    return select """

Hashtags filtradas: apenas aquelas que contenham alguma Hashtag frequente

Segunda opçao, colocando todas as hashtags minúsculas (testando o resultado da comparaçao)

In [ ]:
all_hashtags_lower = [[h.lower() for h in line] for line in all_hashtags]

In [ ]:
len(all_hashtags_lower)

In [ ]:
select = select_hashtag(list_freq, all_hashtags_lower)


In [ ]:
len(select)

Aplica o algoritmo apriori, que retorna os itens frequentes na lista de hashtags. O parâmetro min_suport (0-1) "regula" o quanto um item é considerado "frequente" na lista.

In [ ]:
te = TransactionEncoder()
te_ary = te.fit(select).transform(select)
df = pd.DataFrame(te_ary, columns=te.columns_)
frequent_itemsets = apriori(df, min_support=0.2, use_colnames=True)

frequent_itemsets

Aplicando associação usando "metric=confidence" (Fornece a probabilidade do consequente em uma transação, dada a presença de antecedente)

In [ ]:
#rules=association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

Aplicando associação usando "metric=lift" (Dado que antecedentes e consequentes são independentes, com que frequência eles se juntam)

In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.antecedents = rules.antecedents.apply(lambda x: next(iter(x)))
rules.consequents = rules.consequents.apply(lambda x: next(iter(x)))
#rules

Usando NetworkX para construir um gráfico para verificar a associação entre antecedentes e consequentes obtidos após a regra de associação aplicada.

In [ ]:
fig, ax=plt.subplots(figsize=(16,9))
GA=nx.from_pandas_edgelist(rules,source='antecedents',target='consequents')
nx.draw(GA,with_labels=True)

plt.show()

Testando os parâmetros para melhorar a visualização

https://networkx.github.io/documentation/networkx-1.10/reference/drawing.html

In [ ]:
fig, ax=plt.subplots(figsize=(16,9))

GA=nx.from_pandas_edgelist(rules,source='antecedents',target='consequents')

pos = nx.spring_layout(GA)

nx.draw_networkx(GA,pos,with_labels=False)

nx.draw_networkx_edges(GA, pos, edgelist=None, width=2.0, edge_color='R', style='solid', alpha=1.0, edge_cmap=None, edge_vmin=None, edge_vmax=None, ax=None, arrows=True, label=None)

#nx.draw_networkx_labels(GA, pos, labels=None, font_size=12, font_color='k', font_family='sans-serif', font_weight='normal', alpha=1.0, bbox=None, ax=None)

nx.draw_networkx_nodes(GA, pos, nodelist=None, node_size=100, node_color='y', node_shape='o', alpha=1.0, cmap=None, vmin=None, vmax=None, ax=None, linewidths=None, label=None)

text = nx.draw_networkx_labels(GA,pos)

for _,t in text.items():
    t.set_rotation('horizontal')
    

    